In [49]:
"""Volume 2: Non-negative Matrix Factorization."""

import numpy as np
import cvxpy as cp
from matplotlib import pyplot as plt
import os
from imageio import imread
import warnings
warnings.filterwarnings("ignore")
from IPython.display import clear_output

from sklearn.decomposition import NMF
from sklearn.metrics import mean_squared_error as mse

In [50]:
#Problems 1-2
class NMFRecommender:

    def __init__(self,random_state=15,rank=3,maxiter=200,tol=1e-3):
        """The parameter values for the algorithm"""
        self.random_state = random_state
        self.rank = rank
        self.maxiter = maxiter
        self.tol = tol

        
    
    def initialize_matrices(self,m,n):
        """randomly initialize the W and H matrices,"""
        np.random.seed(self.random_state)
        self.W = np.random.random((m,self.rank))
        self.H = np.random.random((self.rank,n))
        return self.W, self.H

      
    def fit(self,V):
        """Fits W and H weight matrices using CVXPY"""
        def solve_W():
            new_W = cp.Variable(self.W.shape, nonneg = True)
            objW = cp.Minimize(cp.norm(V-new_W@self.H, 'fro'))
            probW = cp.Problem(objW)
            probW.solve()
            return new_W.value

        def solve_H():
            new_H = cp.Variable(self.H.shape, nonneg = True)
            objH = cp.Minimize(cp.norm(V-self.W@new_H, 'fro'))
            probH = cp.Problem(objH)
            probH.solve()
            return new_H.value

        for i in range(self.maxiter):
            print(f'iteration {i}')
            # print("old W:")
            # print(self.W)
            self.W = solve_W()
            # print("new W:")
            # print(self.W)

            # print("old H:")
            # print(self.H)
            self.H = solve_H()
            # print("new H:")
            # print(self.H)
            print(f"norm: {np.linalg.norm(V-self.W@self.H, ord='fro')}")
            if np.linalg.norm(V-self.W@self.H, ord='fro') <= self.tol:
                break
        print(f'solved in {i} iterations')


    def reconstruct(self):
        """Reconstruct V matrix for comparison against the original V"""
        return self.W@self.H

        raise NotImplementedError('Problem 2 incomplete')


In [51]:
V = np.array([	[0,1,0,1,2,2],
				[2,3,1,1,2,2],
				[1,1,1,0,1,1],
				[0,2,3,4,1,1],
				[0,0,0,0,1,0]])
m,n = V.shape
WHsolver = NMFRecommender()
W,H = WHsolver.initialize_matrices(m,n)

WHsolver.fit(V)

# new_W = cp.Variable(W.shape, nonneg = True)
# new_H = cp.Variable(H.shape, nonneg = True)
# objective1 = cp.Minimize(cp.norm(V-new_W@H, "fro"))
# objective2 = cp.Minimize(cp.norm(V-W@new_H, 'fro'))

# problem1 = cp.Problem(objective1)
# problem2 = cp.Problem(objective2)

# print("old W:")
# print(W)

# problem1.solve()
# W = new_W.value
# print("old H:")
# print(H)
# problem2.solve()
# H = new_H.value

# print("W")
# print(W)
# print("H")
# print(H)


iteration 0
norm: 2.8370757890343397
iteration 1
norm: 1.7527138396989856
iteration 2
norm: 1.5548074898579909
iteration 3
norm: 1.411545268954689
iteration 4
norm: 1.2762973499457915
iteration 5
norm: 1.145664511063516
iteration 6
norm: 1.0763108569599196
iteration 7
norm: 1.0539080741912785
iteration 8
norm: 1.0487540034167655
iteration 9
norm: 1.0475230403560816
iteration 10
norm: 1.0470921506660453
iteration 11
norm: 1.046830151042194
iteration 12
norm: 1.0466098248720708
iteration 13
norm: 1.0464053101281408
iteration 14
norm: 1.0462110631210026
iteration 15
norm: 1.0460256542308386
iteration 16
norm: 1.0458485488280416
iteration 17
norm: 1.045679398165735
iteration 18
norm: 1.0455179080276769
iteration 19
norm: 1.045363792060489
iteration 20
norm: 1.0452167859291535
iteration 21
norm: 1.0450766329138907
iteration 22
norm: 1.044943082279431
iteration 23
norm: 1.0448158881761256
iteration 24
norm: 1.0446948087035508
iteration 25
norm: 1.0445796068743674
iteration 26
norm: 1.0444700

In [ ]:
def prob3():
    """Run NMF recommender on the grocery store example"""
    V = np.array(
        [
            [0, 1, 0, 1, 2, 2],
            [2, 3, 1, 1 ,2, 2],
            [1, 1, 1, 0, 1, 1],
            [0, 2, 3, 4, 1, 1],
            [0, 0, 0, 0, 1, 0]
        ]
    )

    raise NotImplementedError("problem 3 not implemented")

In [3]:

#get data
def get_faces(path="./faces94"):
    """Traverse the specified directory to obtain one image per subdirectory.
    Flatten and convert each image to grayscale.

    Parameters:
        path (str): The directory containing the dataset of images.

    Returns:
        ((mn,k) ndarray) An array containing one column vector per
            subdirectory. k is the number of people, and each original
            image is mxn.
    """
    # Traverse the directory and get one image per subdirectory.
    faces = []
    for (dirpath, dirnames, filenames) in os.walk(path):
        for fname in filenames:
            if fname[-3:]=="jpg":       # Only get jpg images.
                # Load the image, convert it to grayscale,
                # and flatten it into a vector.
                faces.append(np.ravel(imread(dirpath+"/"+fname, as_gray=True)))
                break
    # Put all the face vectors column-wise into a matrix.
    return np.transpose(faces)

def show(image, m=200, n=180, plt_show=False):
    """Plot the flattened grayscale 'image' of width 'w' and height 'h'.

    Parameters:
        image ((mn,) ndarray): A flattened image.
        m (int): The original number of rows in the image.
        n (int): The original number of columns in the image.
        plt_show (bool): if True, call plt.show() at the end
    """
    #scale image
    image = image / 255
    #reshape image
    image = np.reshape(image,(m,n))
    #show image
    plt.imshow(image,cmap = "gray")
    
    if plt_show:
        plt.show()


In [4]:
def prob4():
    """
        Gridsearch over rank, alpha and l1_ratio values to reconstruct 
        image using NMF. Plot all reconstructed images.
    """
    raise NotImplementedError("Problem 3 incomplete")


In [ ]:
def prob5():
    '''
        find the 10 basis faces with the largest coefficients 
        corresponding to the the second and twelfth face in the dataset. 
        Plot these basis faces along with the original image using 
        subplots
    '''
    raise NotImplementedError("Problem 4 incomplete")